# Build PacBio consensus sequences
This notebook builds consensus sequences for each barcode from the PacBio CCS sequencing.

Import Python modules:

In [1]:
import os
import itertools

import Bio.SeqIO
import alignparse.utils
import alignparse.consensus
import dms_variants.barcodes

import yaml
import numpy
import scipy.stats
import pandas as pd
import altair as alt

In [2]:
_ = alt.data_transformers.disable_max_rows()

In [3]:
# Set path prefix to "../" if running locally and "" if running with Snakemake
path_prefix = ""

Get configuration information:

In [4]:
# If you are running notebook interactively rather than in pipeline that handles
# working directories, you may have to first `os.chdir` to appropriate directory.

with open(path_prefix + "config.yaml") as f:
    config = yaml.safe_load(f)

Get the aligned CCSs:

In [7]:
aligned_ccs = pd.read_csv(
    path_prefix + "results/CCSs_aligned_to_amplicon.csv",
    na_filter=None,
)

aligned_ccs.head()

/tmp/ipykernel_2761/4084475139.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  aligned_ccs = pd.read_csv(


,query_name,gene_mutations,gene_accuracy,pacbioRun,library,barcode,barcode_accuracy
0,m64272e_240719_173010/37/ccs,A232T G233T A234C,1.000000,E3E2_Lib_A_240719,E3E2_Lib_A,,
1,m64272e_240719_173010/39/ccs,G520T A521G,1.000000,E3E2_Lib_A_240719,E3E2_Lib_A,,
2,m64272e_240719_173010/44/ccs,G358C G359T C360G G2588A,1.000000,E3E2_Lib_A_240719,E3E2_Lib_A,,
3,m64272e_240719_173010/99/ccs,G544T A545T,1.000000,E3E2_Lib_A_240719,E3E2_Lib_A,,
4,m64272e_240719_173010/108/ccs,C178T C179G T180C del1680to1680 del2493to2493,0.999713,E3E2_Lib_A_240719,E3E2_Lib_A,,


Get the different types of library design.

In [11]:
design_df = pd.read_csv(path_prefix + config['runs'])[['library', 'design']]
design_df.head()

,library,design
0,E3E2_Lib_A,Plasmid_Pool
1,6KE1_Lib_B,Plasmid_Pool
2,E3E2_BCPCR_A,Barcode_PCR
3,6KE1_BCPCR_B,Barcode_PCR
4,E3E2_1_GS,Genscript_Product


Although there are 3 types of design, the gene sequence should be the same between them.

In [21]:
geneseqs = {}
for design in design_df.design.unique():
    geneseqs[design] = str(Bio.SeqIO.read(path_prefix + f"results/gene_sequence/{design}_codon.fasta", "fasta").seq)
 

# Make sure all gene sequences are the same length
assert len({len(seq) for seq in geneseqs.values()}) == 1

# Assign the first sequence to the variable `geneseq` for use in the rest of the notebook
geneseq = next(iter(geneseqs.values()))

Some of the libraries (the Plasmid Pool and GenScript Product) don't have barcodes from `alignparse`. There are no barcodes in the GenScript Product and the Plasmid Pool was digested too close to the end of the barcode to align properly.

Below, I'll add an artificial barcode to both libraries (for now) and an accuracy of 1.0

In [40]:
genscript_libraries = design_df[design_df.design == "Genscript_Product"].library.unique().tolist()
plasmid_libraries = design_df[design_df.design == "Plasmid_Pool"].library.unique().tolist()
missing_barcode_libraries = plasmid_libraries + genscript_libraries
print("Adding barcodes to libraries:", missing_barcode_libraries)

aligned_ccs.loc[aligned_ccs.library.isin(missing_barcode_libraries), 'barcode'] = [f"barcode_{i}" for i in range(aligned_ccs[aligned_ccs.library.isin(missing_barcode_libraries)].shape[0])]
aligned_ccs.loc[aligned_ccs.library.isin(missing_barcode_libraries), 'barcode_accuracy'] = 1.0

# Coerce the barcode_accuracy column to be numeric
aligned_ccs['barcode_accuracy'] = pd.to_numeric(aligned_ccs['barcode_accuracy'], errors='coerce')

Adding barcodes to libraries: ['E3E2_Lib_A', '6KE1_Lib_B', 'E3E2_1_GS', '6KE1_2_GS']


## Filter CCSs for accuracy

Plot the gene and barcode accuracy for each CCS, and only keep those above an accuracy threshold:

In [41]:
max_error_rate = config["max_ccs_error_rate"]  # require error rate <= this to keep CCS

log10_error_floor = -7  # error rates less than this set to this for plotting
log10_error_ceil = -2  # error rates greater than this set to this for plotting
nbins = 100  # bins for cumulative fraction plot

# calculate error rates
aligned_ccs = aligned_ccs.assign(
    gene_error=lambda x: 1 - x["gene_accuracy"],
    barcode_error=lambda x: 1 - x["barcode_accuracy"],
)

# calculate cumulative frequencies on log error rates
cumfrac = (
    aligned_ccs.melt(
        id_vars=["query_name", "pacbioRun"],
        value_vars=["barcode_error", "gene_error"],
        value_name="error",
    )
    .assign(log10_error=lambda x: numpy.log10(x["error"]).clip(lower=log10_error_floor))
    .groupby(["variable", "pacbioRun"])
    .apply(
        lambda g: pd.DataFrame(
            {
                "cumulative_count": scipy.stats.cumfreq(
                    g["log10_error"],
                    numbins=nbins,
                    defaultreallimits=(log10_error_floor, log10_error_ceil),
                )[0],
                "log10_error": numpy.linspace(
                    log10_error_floor, log10_error_ceil, nbins
                ),
            }
        )
    )
    .assign(
        meets_accuracy_cutoff=lambda x: x["log10_error"] <= numpy.log10(max_error_rate),
        cumulative_fraction=lambda x: x["cumulative_count"] / len(aligned_ccs),
    )
    .reset_index()
)

# plot cumulative frequencies
cumfrac_chart = (
    alt.Chart(cumfrac)
    .encode(
        x=alt.X(
            "log10_error",
            scale=alt.Scale(zero=False),
        ),
        y="cumulative_count",
        color="meets_accuracy_cutoff",
        column=alt.Column("variable", title=None),
        row=alt.Row("pacbioRun", title=None),
        tooltip=[
            alt.Tooltip("log10_error", format=".3f"),
            alt.Tooltip("cumulative_fraction", format=".3f"),
            alt.Tooltip("cumulative_count", format=".4g"),
        ],
    )
    .mark_point(filled=True, size=30)
    .properties(width=225, height=120)
)
display(cumfrac_chart)

/home/whannon/miniforge3/envs/dms-vep-pipeline-3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_2761/2211512034.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


alt.Chart(...)

Remove CCSs that do not meet accuracy threshold:

In [42]:
print(f"There are {len(aligned_ccs)} CCSs before accuracy filtering.")
aligned_ccs = aligned_ccs.query("barcode_error <= @max_error_rate").query(
    "gene_error <= @max_error_rate"
)
print(f"After filtering {len(aligned_ccs)} CCSs remain.")

There are 3346755 CCSs before accuracy filtering.
After filtering 1859548 CCSs remain.


## Convert in-frame deletions to substitutions
If a deletion is in-frame, convert to substitution format using `-` as the substitution character:

In [43]:
deltosubs = alignparse.utils.InFrameDeletionsToSubs(geneseq)

aligned_ccs = (
    aligned_ccs.assign(
        mutations_inframe=lambda x: x["gene_mutations"].map(deltosubs.dels_to_subs),
        inframe_deletion=lambda x: x["gene_mutations"] != x["mutations_inframe"],
    )
    .drop(columns="gene_mutations")
    .rename(columns={"mutations_inframe": "gene_mutations"})
)

How many CCSs have in-frame deletions?

In [44]:
print("Number of CCSs with in-frame deletions:")
display(
    aligned_ccs.groupby("inframe_deletion")
    .aggregate(n_CCSs=pd.NamedAgg("query_name", "count"))
    .assign(fraction=lambda x: x["n_CCSs"] / x["n_CCSs"].sum())
    .round(3)
)

Number of CCSs with in-frame deletions:


,n_CCSs,fraction
inframe_deletion,,
False,1851562,0.996
True,7986,0.004


## Empirical accuracy of CCSs
We can compute the empirical accuracy of individual CCSs by comparing mutations found in CCSs with the same barcode.

First, annotate which CCSs have indels (not in frame):

In [45]:
aligned_ccs = alignparse.consensus.add_mut_info_cols(
    aligned_ccs,
    mutation_col="gene_mutations",
    n_indel_col="n_indels",
    overwrite_cols=True,
).assign(has_indel=lambda x: x["n_indels"] > 0)

Now compute empirical accuracy, for all sequences and only those without indels, and at the error-rate filter we chose to use and one 10-fold higher.
Note that the in-frame codon deletions are not classified as deletions since we have re-classified as substitutions above.
The empirical accuracy is the estimated accuracy of each **individual** CCS:

## Build consensus sequences
Use the [alignparse.consensus.simple_mutconsensus](https://jbloomlab.github.io/alignparse/alignparse.consensus.html#alignparse.consensus.simple_mutconsensus) method to build consensus sequences, and plot how many barcodes and CCSs contributed to valid consensuses or had to be dropped.
Note the stats for barcodes and CCSs look different, because there are uneven numbers of CCSs per barcode:

In [49]:
# get parameters for building consensus
consensus_params = config["consensus_params"]
print(
    "Building consensus sequences with following settings\n  "
    + "\n  ".join(f"{param}={val}" for param, val in consensus_params.items())
)

# build consensus sequences and plot results
max_plot_nseqs = 15  # group nseqs >= this together
plot_width = 225

consensus, dropped = alignparse.consensus.simple_mutconsensus(
    aligned_ccs, mutation_col="gene_mutations", **consensus_params
)

consensus_stats = (
    pd.concat(
        [
            consensus.rename(columns={"variant_call_support": "nseqs"})
            .drop(columns="gene_mutations")
            .assign(drop_reason="retained", dropped=False),
            dropped.assign(dropped=True),
        ]
    )
    .assign(nseqs=lambda x: x["nseqs"].clip(upper=max_plot_nseqs))
    .groupby(["library", "drop_reason", "dropped", "nseqs"], as_index=False)
    .aggregate(
        n_barcodes=pd.NamedAgg("barcode", "count"),
        n_CCSs=pd.NamedAgg("nseqs", "sum"),
    )
    .rename(columns={"drop_reason": "category"})
    .melt(
        id_vars=["library", "category", "dropped", "nseqs"],
        value_vars=["n_barcodes", "n_CCSs"],
        var_name="type_of_count",
        value_name="count",
    )
)

# get drop reasons in order to plot
drop_reasons = (
    consensus_stats.groupby(["category", "dropped"], as_index=False)
    .aggregate({"count": "sum"})
    .sort_values(["dropped", "count"], ascending=[True, False])["category"]
    .tolist()
)
drop_colors = ["blue", "orange", "orangered", "goldenrod", "gold", "darkorange"]
assert len(drop_reasons) <= len(drop_colors)

consensus_stats_chart = (
    alt.Chart(consensus_stats)
    .encode(
        x=alt.X(
            "nseqs",
            scale=alt.Scale(domain=(1, max_plot_nseqs)),
            title="number of CCSs for barcode",
        ),
        y=alt.Y("count", stack=True),
        color=alt.Color(
            "category",
            sort=drop_reasons,
            scale=alt.Scale(range=drop_colors),
        ),
        row=alt.Row(
            "library",
            title=None,
        ),
        column=alt.Column("type_of_count"),
        tooltip=consensus_stats.columns.tolist(),
    )
    .mark_bar(size=0.75 * plot_width / max_plot_nseqs)
    .properties(width=plot_width, height=120)
    .configure_axis(grid=False)
    .resolve_scale(y="independent")
)

display(consensus_stats_chart)

Building consensus sequences with following settings
  max_sub_diffs=None
  max_indel_diffs=None
  max_minor_sub_frac=1
  max_minor_indel_frac=1
  min_support=1


alt.Chart(...)

## Write consensus sequences to barcode-variant lookup tables for subsequent use
We filter any consensus sequences with indels that are not in-frame codon-length deletions (so filtering out-of-frame indels), and then write the remaining sequences to barcode-variant lookup tables for later use:

In [50]:
# annotate with information on mutation types
consensus = alignparse.consensus.add_mut_info_cols(
    consensus,
    mutation_col="gene_mutations",
    sub_str_col="substitutions",
    n_indel_col="n_indels",
)

# plot summary stats
stats = (
    consensus.assign(
        out_of_frame_indel=lambda x: x["n_indels"] > 0,
        in_frame_codon_deletion=lambda x: (
            (~x["out_of_frame_indel"]) & (x["substitutions"].str.contains("-"))
        ),
        no_indel=lambda x: (~x["out_of_frame_indel"]) & (~x["in_frame_codon_deletion"]),
    )[["library", "out_of_frame_indel", "in_frame_codon_deletion", "no_indel"]]
    .groupby("library", as_index=False)
    .aggregate("sum")
    .melt(
        id_vars="library",
        var_name="category",
        value_name="n_barcodes",
    )
    .assign(retained=lambda x: x["category"] != "out_of_frame_indel")
)
stats_chart = (
    alt.Chart(stats)
    .encode(
        x=alt.X("n_barcodes"),
        y=alt.Y("category", title=None),
        color=alt.Color("retained"),
        tooltip=stats.columns.tolist(),
        facet=alt.Facet("library", columns=2),
    )
    .mark_bar()
    .properties(width=175, height=45)
)
display(stats_chart)

# write to file
consensus = consensus.query("n_indels == 0")
nt_variants = path_prefix + "results/variants/nt_variants.csv"
os.makedirs(os.path.dirname(nt_variants), exist_ok=True)
_ = consensus.to_csv(
    consensus[["library", "barcode", "substitutions", "variant_call_support"]].to_csv(
        nt_variants,
        index=False,
    )
)

alt.Chart(...)

Writing 1502414 consensus sequences to ../results/variants/nt_variants.csv
